## Otimização de hiperparâmetros



## Introdução



Escolheu-se "árvores aleatórias" como algoritmo para otimizar três hiperparâmetros utilizando-se de redes neurais para achar o melhor conjunto de hiperparâmetros.


## Objetivo



**Objetivo**: use algoritmos genéticos para encontrar um bom conjunto de hiperparâmetros em um experimento de aprendizado de máquina. Escolha um algoritmo que tenha pelo menos 3 hiperparâmetros para serem otimizados.



## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from itertools import product

## Códigos e discussão



-   Use células de código para o código.

-   Use células de texto para a discussão.

-   A discussão não deve ser feita em comentários dentro das células de código. Toda discussão deve acontecer após o resultado sendo discutido foi apresentado. Exemplo: não discuta um gráfico antes de apresentá-lo.



In [ ]:
######################################
# CÓDIGO INCOMPLETO
######################################
#O QUE PRECISA SER FEITO:
#DEFINIR UM JEITO MAIS EFICIENTE DE CRIAR GENES
#DEFINIR A FUNCAO MUTACAO
#VER SE O CODIGO RODA

######################################

In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 40
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3
#NUM_GERACOES = 30
NUM_GENES = 5

# relacionadas ao problema a ser resolvido - dataset
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 1024
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]


# relacionadas ao problema a ser resolvido - hiperparametros
#irei variar "NUM_FOLHAS, NUM_PROFUNDIDADE, MINIMO_FOLHAS"
#deixei aqui o resto para demonstrar o que se poderia variar. o resto está nos valores padrão
#NUM_FOLHAS = 50 #max_leaf_nodes
#NUM_PROFUNDIDADE = 70 #max_depth
#CRITERIO = 'absolute_error'
SPLITTER = "best"
#MINIMO_SPLIT = 2 #min_samples_split
MINIMO_FOLHAS = 1
PESO_FOLHAS = 0.0
MAXIMO_FEATURES = None
IMPUREZA_MINIMO = 0.0
COMPLEXO_ALPHA = 0.0


In [3]:
df = sns.load_dataset(DATASET_NAME) #criação do dataset
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [4]:
tamanho = 0.25 # Fração de dados escolhida para treino e teste
seed = SEMENTE_ALEATORIA

i = df.index
i_treino, i_teste = train_test_split(i, test_size=tamanho, random_state = seed)

df_treino = df.loc[i_treino]
df_teste = df.loc[i_teste]

In [5]:
X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values
X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values

In [6]:
def gene_hiperparametros(min_hiperparametro, max_hiperparametro):
    '''Gera um valor aleatório para cada hiperparâmetro em uma dada faixa.
    
    Returns:
        Um valor válido para os 3 hiparparâmetros variados
        '''
    if min_hiperparametro <= 0:
        return 'o valor minimo do hiperparametro não pode ser menor ou igual a zero'
    
    lista_faixa_hiperparametro = range(min_hiperparametro, max_hiperparametro+1) #anula o -1 do range
    gene = random.choice(lista_faixa_hiperparametro)

    return gene

In [7]:
def individuo_hiperparametros(quantidade_hiperparametros,min_hiperparametro,max_hiperparametro):
    candidato = []

    for _ in range(0,quantidade_hiperparametros):
        candidato.append(gene_hiperparametros(min_hiperparametro,max_hiperparametro))

    return candidato

In [8]:
def populacao_inicial_hiperparametros(tamanho, quantidade_hiperparametros,min_hiperparametro,max_hiperparametro): 

    populacao = []
    for _ in range(tamanho):
        populacao.append(individuo_hiperparametros(quantidade_hiperparametros,min_hiperparametro,max_hiperparametro)) #cria uma lista de listas como populacao
    return populacao


In [9]:
def selecao_torneio_min(populacao, fitness, tamanho_torneio=3): #implementada em funcoes.py
    """Faz a seleção de uma população usando torneio.

    Nota: da forma que está implementada, só funciona em problemas de
    minimização.

    Args:
      populacao: população do problema
      fitness: lista com os valores de fitness dos individuos da população
      tamanho_torneio: quantidade de invidiuos que batalham entre si

    Returns:
      Individuos selecionados. Lista com os individuos selecionados com mesmo
      tamanho do argumento `populacao`.
    """
    selecionados = []

    # criamos essa variável para associar cada individuo com seu valor de fitness
    par_populacao_fitness = list(zip(populacao, fitness))

    # vamos fazer len(populacao) torneios! Que comecem os jogos!
    for _ in range(len(populacao)):
        combatentes = random.sample(par_populacao_fitness, tamanho_torneio)

        # é assim que se escreve infinito em python
        minimo_fitness = float("inf")

        for par_individuo_fitness in combatentes:
            individuo = par_individuo_fitness[0]
            fit = par_individuo_fitness[1]

            # queremos o individuo de menor fitness
            if fit < minimo_fitness:
                selecionado = individuo
                minimo_fitness = fit

        selecionados.append(selecionado)

    return selecionados


In [10]:
def cruzamento_ordenado(pai, mae): #implementado em funcoes.py
    """Operador de cruzamento ordenado.

    Neste cruzamento, os filhos mantém os mesmos genes que seus pais tinham,
    porém em uma outra ordem. Trata-se de um tipo de cruzamento útil para
    problemas onde a ordem dos genes é importante e não podemos alterar os genes
    em si. É um cruzamento que pode ser usado no problema do caixeiro viajante.

    Ver pág. 37 do livro do Wirsansky.

    Args:
      pai: uma lista representando um individuo
      mae : uma lista representando um individuo

    Returns:
      Duas listas, sendo que cada uma representa um filho dos pais que foram os
      argumentos. Estas listas mantém os genes originais dos pais, porém altera
      a ordem deles
    """
    corte1 = random.randint(0, len(pai) - 2)
    corte2 = random.randint(corte1 + 1, len(pai) - 1)
    
    filho1 = pai[corte1:corte2]
    for gene in mae:
        if gene not in filho1:
            filho1.append(gene)
            
    filho2 = mae[corte1:corte2]
    for gene in pai:
        if gene not in filho2:
            filho2.append(gene)
            
    return filho1, filho2

In [11]:
def funcao_objetivo_standard():

    y_verdadeiro = y_teste
    y_previsao = modelo_dt.predict(X_teste)

    MSE_standard = mean_squared_error(y_verdadeiro, y_previsao, squared=True)

    return MSE_standard

In [12]:
def funcao_objetivo_hiperparametros(individuo):
    num_folhas = individuo[0]
    num_profundidade = individuo[1]
    min_samples_folhas = individuo[2]
    
    modelo_dt = DecisionTreeRegressor(
        max_leaf_nodes=num_folhas,
        max_depth=num_profundidade,
        #criterion=CRITERIO,
        #splitter=SPLITTER,
        #min_samples_split=MINIMO_SPLIT,
        min_samples_leaf=MINIMO_FOLHAS,
        #min_weight_fraction_leaf=PESO_FOLHAS,
        #max_features=MAXIMO_FEATURES,
        #min_impurity_decrease=IMPUREZA_MINIMO,
        #ccp_alpha=COMPLEXO_ALPHA,
        random_state=1024,
    )

    modelo_dt.fit(X_treino, y_treino)

    y_verdadeiro = y_teste
    y_previsao = modelo_dt.predict(X_teste)
    MSE = mean_squared_error(y_verdadeiro, y_previsao, squared=True)

    MSE_standard = funcao_objetivo_standard()
    
    fitness = 0

    if MSE < MSE_standard:
        fitness += 500
        
    else:
        fitness +=0


    return fitness

In [15]:
def funcao_objetivo_pop_hiperparametros(populacao):

    #print(populacao)
    fitness_pop = []
    for individuo in populacao:
        fitness_individuo = funcao_objetivo_palindromo(individuo)
        
        fitness_pop.append(fitness_individuo)
    return fitness_pop


In [17]:
# funções locais

def cria_populacao_inicial(tamanho_pop,num_genes)

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_hiperparametros(populacao)

def funcao_mutacao(individuo):
    return mutacao_espelhada_palindromo(individuo, LETRAS_POSSIVEIS)

In [18]:
populacao = cria_populacao_inicial(TAMANHO_POP, NUM_GENES,LETRAS_POSSIVEIS)
set_palindromos = set()

while len(set_palindromos) != QUANTIDADE_PALINDROMOS:
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador + 2   
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)            
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    if menor_fitness == 0:       
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = ''.join(populacao[posicao]) #list is unhasheable
        set_palindromos.add(melhor_individuo_ja_visto)

print()
print(f'Aqui estão {QUANTIDADE_PALINDROMOS} palindromos diferentes:')
print(set_palindromos)

NameError: name 'cria_populacao_inicial' is not defined

## Conclusão



Delete este texto e escreva sua conclusão.



## Referências consultadas



1.  Delete este texto e inclua suas referências ordenadas numericamente. Se for referenciar no notebook, use o número entre colchetes (exemplo: para citar essa referência aqui escreva &ldquo;[1]&rdquo; sem as áspas).

2.  Cada item deve ser numerado. Siga o padrão apresentado.

3.  Caso não tenha nenhuma referência consultada, delete esta seção e o texto contido nela!



## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.

